In [1]:
import os
import random
from tqdm import tqdm
import numpy as np
import time
import logging
import sys
from copy import deepcopy
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, confusion_matrix, matthews_corrcoef
import csv
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
from torch.utils.data.dataset import Dataset
import torchvision.models as models
from torchvision.models import AlexNet_Weights
import torchmetrics
import torchvision.transforms as transforms
from torchvision.transforms import Compose 
from torch.utils.tensorboard import SummaryWriter
torch.backends.cudnn.benchmark=True
print(sys.executable)
print(sys.version)
print(torch.cuda.is_available())
print(f'Torchvision version: {torchvision.__version__}')


c:\Users\jizhi\.conda\envs\flv1\python.exe
3.8.18 (default, Sep 11 2023, 13:39:12) [MSC v.1916 64 bit (AMD64)]
True
Torchvision version: 0.16.0


In [2]:
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_value = 42
set_seed(seed_value)

In [3]:
# set logger
# create logger
logger = logging.getLogger(__name__)
# set log level for all handlers to debug
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
# best for development or debugging
consoleHandler = logging.StreamHandler()
consoleHandler.setLevel(logging.DEBUG)

# create formatter
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(asctime)s-%(levelname)s: %(message)s')

# add formatter to ch
consoleHandler.setFormatter(formatter)

# add ch to logger
logger.addHandler(consoleHandler)

In [4]:
# prepare image
train_names_list = ['hospital','sim-room','warehouse']

val_names_list = ['hospital','sim-room','warehouse','overall']

train_data_folders_list = [ 'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\hospital\\train',
                            'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\sim-room\\train',
                            'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\warehouse\\train',
                    ]

val_data_folders_list = [ 'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\hospital\\test',
                            'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\sim-room\\test',
                            'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\warehouse\\test',
                            'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\overall\\test',
                    ]

transformed_train_datasets, transformed_val_datasets = [], []

################################################  Data Loaders  #####################################
# 在这个部分，为训练和验证的图像数据定义了一些预处理步骤，包括颜色抖动、重新调整大小、转换为张量和归一化。
# 这些预处理步骤通过 PyTorch 的 transforms 模块实现，并且它们将在训练过程中应用到每个图像上。
for train_dataset in train_data_folders_list:
    transformed_train_datasets.append(
        datasets.ImageFolder(
            train_dataset,
            transforms.Compose([
                transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        )
    ) 

for val_dataset in val_data_folders_list:
    transformed_val_datasets.append(
        datasets.ImageFolder(
            val_dataset,
            transforms.Compose([
                transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        )
    ) 

#########  Data Loaders  ##########

train_loader_list, val_loader_list = [], []

for train_d in transformed_train_datasets:
    train_loader_list.append(
        torch.utils.data.DataLoader(
                train_d,
                batch_size=16,
                shuffle=True,
                num_workers=4,
                drop_last = False,
            )
    )

for val_d in transformed_val_datasets:
    val_loader_list.append(
        torch.utils.data.DataLoader(
                val_d,
                batch_size=1,
                shuffle=True,
                num_workers=4,
                drop_last = False,
            )
    )


In [5]:
# Compute ROC curve and ROC area for each class
print("---------------- Evaluation  Started ---------------")

# Create or open a CSV file to store the results
csv_file_path = "evaluation_results.csv"
with open(csv_file_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write header
    csv_writer.writerow(['Model Name', 'AUC','Accuracy', 'Precision', 'Recall', 'F1-Score', 'Matthews Correlation Coefficient'])

device = torch.device('cuda')
# 指定文件夹路径
folder_path = 'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_results_backup_epoch50center\\saved_models\\sim\\fusion_models'

# 获取文件夹中所有文件的名称
file_names = os.listdir(folder_path)

# 筛选出以 .pth 结尾的文件
pth_names = [f for f in file_names if f.endswith('.pth')]

# 打印出所有以 .pth 结尾的文件的完整路径
for pth_name in pth_names:
    pth_file = os.path.join(folder_path, pth_name)
    model_entity = models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
    model_entity.classifier[6] = torch.nn.Linear(model_entity.classifier[6].in_features, 2)

    model_entity.load_state_dict(torch.load(pth_file))
    model_entity.to(device)
    # 设置模型为评估模式
    model_entity.eval()

    for val_inx, val_loader in enumerate(val_loader_list):
        if val_inx == 3:
            label_lists = []
            prediction_lists = []
            pos_prob_lists = []

            with torch.no_grad():
                for image, label in val_loader:
                    image = image.to(device)
                    model_output = model_entity(image)#模型直接输出的是对数几率"（Logits）"，因此我们需要应用 softmax 函数，将其转换为概率。


                    softmax_output = F.softmax(model_output, dim=1)  # 使用softmax函数
                    pos_prob = softmax_output[:, 1].item()  # 取第二列的值作为正例的概率
                    pos_prob = np.float64(pos_prob)

                    prediction= model_output.argmax().item()  # 取概率最大的那一列作为预测值
                    prediction = np.int32(prediction)

                    label = label.numpy()[0]  # 取出label的值#这是因为我的每一个batch_size都是1,所以label.numpy()[0]是一个numpy.int64
                    label = np.int32(label)

                    pos_prob_lists.append(pos_prob)  # 保存正例的概率,这里的pos_prob是一个float
                    label_lists.append(label)
                    prediction_lists.append(prediction)

                label_arrays = np.array(label_lists)
                prediction_arrays = np.array(prediction_lists)
                pos_prob_arrays = np.array(pos_prob_lists)

                # Compute Accuracy
                accuracy = accuracy_score(label_arrays.tolist(), prediction_arrays.tolist())
                precision = precision_score(label_arrays.tolist(), prediction_arrays.tolist())
                recall = recall_score(label_arrays.tolist(), prediction_arrays.tolist())
                f1 = f1_score(label_arrays.tolist(), prediction_arrays.tolist())
                
                # Compute Confusion Matrix
                cm = confusion_matrix(label_arrays.tolist(), prediction_arrays.tolist())
                
                # Compute Matthews Correlation Coefficient
                mcc = matthews_corrcoef(label_arrays.tolist(), prediction_arrays.tolist())


                label_arrays = torch.from_numpy(label_arrays).clone()
                prediction_arrays = torch.from_numpy(prediction_arrays).clone()
                pos_prob_arrays = torch.from_numpy(pos_prob_arrays).clone()

                # Compute ROC curve and ROC area for each class
                roc = torchmetrics.ROC(task="binary")
                fpr, tpr, thresholds = roc(pos_prob_arrays, label_arrays)

                # 使用torchmetrics计算AUC
                auc = torchmetrics.AUROC(task="binary")
                auc_value = auc(pos_prob_arrays, label_arrays)

                # print("fpr",fpr)
                # print("tpr",tpr)
                # print("thresholds",thresholds)

                print(f'AUC: {auc_value:.5f}, Accuracy:{accuracy:.5f}, Precision: {precision:.5f}, Recall: {recall:.5f}, F1-Score: {f1:.5f},Matthews Correlation Coefficient: {mcc:.5f}')
                #print(f'Model {pth_name}, Validation Set {val_inx}, AUC: {auc_value:.5f},Accuracy:{accuracy:.5f}')
                # Write the evaluation results to the CSV file
                # Open the CSV file in append mode and write the results
                with open(csv_file_path, mode='a', newline='') as csv_file:
                    csv_writer = csv.writer(csv_file)
                    csv_writer.writerow([pth_name,f"{auc_value:.5f}", f"{accuracy:.5f}", f"{precision:.5f}", f"{recall:.5f}", f"{f1:.5f}", f"{mcc:.5f}"])
print("-------------------- Evaluation  END ------------------------")

---------------- Evaluation  Started ---------------


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\jizhi/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:23<00:00, 10.3MB/s] 


AUC: 0.99202, Accuracy:0.96394, Precision: 0.95215, Recall: 0.97549, F1-Score: 0.96368,Matthews Correlation Coefficient: 0.92816
AUC: 0.99991, Accuracy:0.99519, Precision: 0.99510, Recall: 0.99510, F1-Score: 0.99510,Matthews Correlation Coefficient: 0.99038
AUC: 0.99711, Accuracy:0.93750, Precision: 0.98901, Recall: 0.88235, F1-Score: 0.93264,Matthews Correlation Coefficient: 0.87966
AUC: 0.99077, Accuracy:0.87500, Precision: 0.79688, Recall: 1.00000, F1-Score: 0.88696,Matthews Correlation Coefficient: 0.77551
-------------------- Evaluation  END ------------------------
